## Importing important libraries

In [1]:
import h2o
import sys
from h2o.estimators.gbm import H2OGradientBoostingEstimator

## Initializing H2O 

In [2]:
h2o.init()

ERROR:h2o:Key init.version_check is not a valid config key


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_101"; Java(TM) SE Runtime Environment (build 1.8.0_101-b13); Java HotSpot(TM) 64-Bit Server VM (build 25.101-b13, mixed mode)
  Starting server from /Users/avkashchauhan/.local/lib/python2.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/x7/331tvwcd6p17jj9zdmhnkpyc0000gn/T/tmpC0lLdh
  JVM stdout: /var/folders/x7/331tvwcd6p17jj9zdmhnkpyc0000gn/T/tmpC0lLdh/h2o_avkashchauhan_started_from_python.out
  JVM stderr: /var/folders/x7/331tvwcd6p17jj9zdmhnkpyc0000gn/T/tmpC0lLdh/h2o_avkashchauhan_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster version:,3.14.0.3
H2O cluster version age:,26 days
H2O cluster name:,H2O_from_python_avkashchauhan_jqfer7
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


## Importing dataset from a public URL

In [3]:
local_url = "https://raw.githubusercontent.com/h2oai/sparkling-water/master/examples/smalldata/prostate/prostate.csv"
df = h2o.import_file(local_url)

Parse progress: |█████████████████████████████████████████████████████████| 100%


## Setting up the feature set and response column for machine learning

In [4]:
y = "CAPSULE"
feature_names = df.col_names
feature_names.remove(y) 
df[y] = df[y].asfactor()

## Splitting loaded dataset into training and validation dataset

In [5]:
df_train, df_valid = df.split_frame(ratios=[0.9])
print(df_train.shape)
print(df_valid.shape)

(337, 9)
(43, 9)


## Setting up H2O GBM Esitmatior with all the GBM configuration

In [6]:
prostate_gbm = H2OGradientBoostingEstimator(model_id = "prostate_gbm",
            ntrees=1000,
            learn_rate=0.5,
            max_depth=20,
            stopping_tolerance=0.001,
            stopping_rounds=2,
            score_each_iteration=True)

## Now starting the machine learning process to build a GBM model

In [7]:
prostate_gbm.train(x = feature_names, y = y, training_frame=df_train, validation_frame=df_valid)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


## Taking a look at our GBM model details and its metrics

In [8]:
prostate_gbm

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  prostate_gbm


ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.036289343297
RMSE: 0.190497620187
LogLoss: 0.170007804527
Mean Per-Class Error: 0.0160045361428
AUC: 0.998865964296
Gini: 0.997731928592
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.487417363665: 


,0,1,Error,Rate
0,199.0,2.0,0.01,(2.0/201.0)
1,3.0,133.0,0.0221,(3.0/136.0)
Total,202.0,135.0,0.0148,(5.0/337.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.4874174,0.9815498,134.0
max f2,0.3921417,0.9854015,140.0
max f0point5,0.5407945,0.9924242,130.0
max accuracy,0.5407945,0.9851632,130.0
max precision,0.9929044,1.0,0.0
max recall,0.3068876,1.0,153.0
max specificity,0.9929044,1.0,0.0
max absolute_mcc,0.5407945,0.9694616,130.0
max min_per_class_accuracy,0.4429842,0.9800995,137.0
max mean_per_class_accuracy,0.4874174,0.9839955,134.0


Gains/Lift Table: Avg response rate: 40.36 %



...

,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0232558,0.9747955,2.5294118,2.5294118,1.0,1.0,0.0588235,0.0588235,152.9411765,152.9411765
,2,0.0232558,0.9667251,0.0,2.5294118,0.0,1.0,0.0,0.0588235,-100.0,152.9411765
,3,0.0465116,0.9610079,2.5294118,2.5294118,1.0,1.0,0.0588235,0.1176471,152.9411765,152.9411765
,4,0.0465116,0.9567387,0.0,2.5294118,0.0,1.0,0.0,0.1176471,-100.0,152.9411765
,5,0.0697674,0.9508915,2.5294118,2.5294118,1.0,1.0,0.0588235,0.1764706,152.9411765,152.9411765
,6,0.1162791,0.9045505,1.2647059,2.0235294,0.5,0.8,0.0588235,0.2352941,26.4705882,102.3529412
,7,0.1627907,0.8380050,2.5294118,2.1680672,1.0,0.8571429,0.1176471,0.3529412,152.9411765,116.8067227
,8,0.2093023,0.7061450,1.2647059,1.9673203,0.5,0.7777778,0.0588235,0.4117647,26.4705882,96.7320261
,9,0.3023256,0.6620303,1.8970588,1.9457014,0.75,0.7692308,0.1764706,0.5882353,89.7058824,94.5701357
,10,0.3953488,0.4060167,1.2647059,1.7854671,0.5,0.7058824,0.1176471,0.7058824,26.4705882,78.5467128



Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2017-10-18 20:27:08,0.014 sec,0.0,0.4906113,0.6744291,0.5,1.0,0.5964392,0.4889944,0.6712211,0.5,1.0,0.6046512
,2017-10-18 20:27:08,0.144 sec,1.0,0.4055513,0.5103564,0.8812189,2.4779412,0.2136499,0.4158593,0.5289692,0.8348416,2.5294118,0.2325581
,2017-10-18 20:27:08,0.185 sec,2.0,0.3529658,0.4127270,0.9403717,2.4779412,0.1483680,0.4092823,0.5089544,0.8110860,2.5294118,0.2558140
,2017-10-18 20:27:08,0.213 sec,3.0,0.3179116,0.3510608,0.9645339,2.4779412,0.1008902,0.3767061,0.4404014,0.8857466,2.5294118,0.1395349
,2017-10-18 20:27:08,0.238 sec,4.0,0.2855319,0.2994444,0.9817274,2.4779412,0.0682493,0.3963400,0.4593334,0.8597285,2.5294118,0.2093023
,2017-10-18 20:27:08,0.264 sec,5.0,0.2584285,0.2571172,0.9900132,2.4779412,0.0504451,0.3958879,0.4515288,0.8438914,2.5294118,0.1860465
,2017-10-18 20:27:08,0.287 sec,6.0,0.2311316,0.2203629,0.9961589,2.4779412,0.0326409,0.3883926,0.4424447,0.8755656,2.5294118,0.2325581
,2017-10-18 20:27:08,0.308 sec,7.0,0.2090731,0.1932407,0.9980977,2.4779412,0.0118694,0.4070353,0.4863497,0.8597285,2.5294118,0.2325581
,2017-10-18 20:27:08,0.327 sec,8.0,0.1904976,0.1700078,0.9988660,2.4779412,0.0148368,0.4022264,0.4839237,0.8710407,2.5294118,0.1860465


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
GLEASON,26.4702721,1.0,0.2966210
PSA,17.1931477,0.6495267,0.1926633
ID,12.6726904,0.4787518,0.1420078
VOL,11.9905567,0.4529820,0.1343640
AGE,10.5484657,0.3985024,0.1182042
DPROS,8.5223379,0.3219588,0.0954998
RACE,1.4897647,0.0562807,0.0166940
DCAPS,0.3521338,0.0133030,0.0039459


## Creating a local file and setting up standard output buffer to be written into the local file

In [9]:
old_target = sys.stdout
f = open('/Users/avkashchauhan/Downloads/model_output.txt', 'w')
sys.stdout = f

## Checking local file contents 

In [10]:
!cat /Users/avkashchauhan/Downloads/model_output.txt

## Pritning model objects details as text 

In [11]:
print("Model summary>>> model_object.show()")
prostate_gbm.show()

,0,1,Error,Rate
0,199.0,2.0,0.01,(2.0/201.0)
1,3.0,133.0,0.0221,(3.0/136.0)
Total,202.0,135.0,0.0148,(5.0/337.0)


metric,threshold,value,idx
max f1,0.4874174,0.9815498,134.0
max f2,0.3921417,0.9854015,140.0
max f0point5,0.5407945,0.9924242,130.0
max accuracy,0.5407945,0.9851632,130.0
max precision,0.9929044,1.0,0.0
max recall,0.3068876,1.0,153.0
max specificity,0.9929044,1.0,0.0
max absolute_mcc,0.5407945,0.9694616,130.0
max min_per_class_accuracy,0.4429842,0.9800995,137.0
max mean_per_class_accuracy,0.4874174,0.9839955,134.0


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0118694,0.9848675,2.4779412,2.4779412,1.0,1.0,0.0294118,0.0294118,147.7941176,147.7941176
,2,0.0207715,0.9773450,2.4779412,2.4779412,1.0,1.0,0.0220588,0.0514706,147.7941176,147.7941176
,3,0.0326409,0.9712222,2.4779412,2.4779412,1.0,1.0,0.0294118,0.0808824,147.7941176,147.7941176
,4,0.0415430,0.9662956,2.4779412,2.4779412,1.0,1.0,0.0220588,0.1029412,147.7941176,147.7941176
,5,0.0504451,0.9592614,2.4779412,2.4779412,1.0,1.0,0.0220588,0.125,147.7941176,147.7941176
,6,0.1008902,0.9296896,2.4779412,2.4779412,1.0,1.0,0.125,0.25,147.7941176,147.7941176
,7,0.1513353,0.9025210,2.4779412,2.4779412,1.0,1.0,0.125,0.375,147.7941176,147.7941176
,8,0.2017804,0.8466621,2.4779412,2.4779412,1.0,1.0,0.125,0.5,147.7941176,147.7941176
,9,0.2997033,0.7399699,2.4779412,2.4779412,1.0,1.0,0.2426471,0.7426471,147.7941176,147.7941176
,10,0.4005935,0.4787404,2.3321799,2.4412309,0.9411765,0.9851852,0.2352941,0.9779412,133.2179931,144.1230937


,0,1,Error,Rate
0,20.0,6.0,0.2308,(6.0/26.0)
1,2.0,15.0,0.1176,(2.0/17.0)
Total,22.0,21.0,0.186,(8.0/43.0)


metric,threshold,value,idx
max f1,0.2050763,0.7894737,20.0
max f2,0.0724216,0.8854167,27.0
max f0point5,0.6669863,0.7692308,11.0
max accuracy,0.2050763,0.8139535,20.0
max precision,0.9828660,1.0,0.0
max recall,0.0724216,1.0,27.0
max specificity,0.9828660,1.0,0.0
max absolute_mcc,0.2050763,0.6373241,20.0
max min_per_class_accuracy,0.2245549,0.7692308,19.0
max mean_per_class_accuracy,0.2050763,0.8257919,20.0


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0232558,0.9747955,2.5294118,2.5294118,1.0,1.0,0.0588235,0.0588235,152.9411765,152.9411765
,2,0.0232558,0.9667251,0.0,2.5294118,0.0,1.0,0.0,0.0588235,-100.0,152.9411765
,3,0.0465116,0.9610079,2.5294118,2.5294118,1.0,1.0,0.0588235,0.1176471,152.9411765,152.9411765
,4,0.0465116,0.9567387,0.0,2.5294118,0.0,1.0,0.0,0.1176471,-100.0,152.9411765
,5,0.0697674,0.9508915,2.5294118,2.5294118,1.0,1.0,0.0588235,0.1764706,152.9411765,152.9411765
,6,0.1162791,0.9045505,1.2647059,2.0235294,0.5,0.8,0.0588235,0.2352941,26.4705882,102.3529412
,7,0.1627907,0.8380050,2.5294118,2.1680672,1.0,0.8571429,0.1176471,0.3529412,152.9411765,116.8067227
,8,0.2093023,0.7061450,1.2647059,1.9673203,0.5,0.7777778,0.0588235,0.4117647,26.4705882,96.7320261
,9,0.3023256,0.6620303,1.8970588,1.9457014,0.75,0.7692308,0.1764706,0.5882353,89.7058824,94.5701357
,10,0.3953488,0.4060167,1.2647059,1.7854671,0.5,0.7058824,0.1176471,0.7058824,26.4705882,78.5467128


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2017-10-18 20:27:08,0.014 sec,0.0,0.4906113,0.6744291,0.5,1.0,0.5964392,0.4889944,0.6712211,0.5,1.0,0.6046512
,2017-10-18 20:27:08,0.144 sec,1.0,0.4055513,0.5103564,0.8812189,2.4779412,0.2136499,0.4158593,0.5289692,0.8348416,2.5294118,0.2325581
,2017-10-18 20:27:08,0.185 sec,2.0,0.3529658,0.4127270,0.9403717,2.4779412,0.1483680,0.4092823,0.5089544,0.8110860,2.5294118,0.2558140
,2017-10-18 20:27:08,0.213 sec,3.0,0.3179116,0.3510608,0.9645339,2.4779412,0.1008902,0.3767061,0.4404014,0.8857466,2.5294118,0.1395349
,2017-10-18 20:27:08,0.238 sec,4.0,0.2855319,0.2994444,0.9817274,2.4779412,0.0682493,0.3963400,0.4593334,0.8597285,2.5294118,0.2093023
,2017-10-18 20:27:08,0.264 sec,5.0,0.2584285,0.2571172,0.9900132,2.4779412,0.0504451,0.3958879,0.4515288,0.8438914,2.5294118,0.1860465
,2017-10-18 20:27:08,0.287 sec,6.0,0.2311316,0.2203629,0.9961589,2.4779412,0.0326409,0.3883926,0.4424447,0.8755656,2.5294118,0.2325581
,2017-10-18 20:27:08,0.308 sec,7.0,0.2090731,0.1932407,0.9980977,2.4779412,0.0118694,0.4070353,0.4863497,0.8597285,2.5294118,0.2325581
,2017-10-18 20:27:08,0.327 sec,8.0,0.1904976,0.1700078,0.9988660,2.4779412,0.0148368,0.4022264,0.4839237,0.8710407,2.5294118,0.1860465


variable,relative_importance,scaled_importance,percentage
GLEASON,26.4702721,1.0,0.2966210
PSA,17.1931477,0.6495267,0.1926633
ID,12.6726904,0.4787518,0.1420078
VOL,11.9905567,0.4529820,0.1343640
AGE,10.5484657,0.3985024,0.1182042
DPROS,8.5223379,0.3219588,0.0954998
RACE,1.4897647,0.0562807,0.0166940
DCAPS,0.3521338,0.0133030,0.0039459


## Transferring standard out details to local file

In [12]:
sys.stdout = old_target

## Checking local file contents again to see what is written to the disk

In [13]:
!cat /Users/avkashchauhan/Downloads/model_output.txt

Model summary>>> model_object.show()
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  prostate_gbm


ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.036289343297
RMSE: 0.190497620187
LogLoss: 0.170007804527
Mean Per-Class Error: 0.0160045361428
AUC: 0.998865964296
Gini: 0.997731928592
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.487417363665: 
Maximum Metrics: Maximum metrics at their respective thresholds

Gains/Lift Table: Avg response rate: 40.36 %



ModelMetricsBinomial: gbm
** Reported on validation data. **

MSE: 0.161786079676
RMSE: 0.402226403505
LogLoss: 0.483923658542
Mean Per-Class Error: 0.174208144796
AUC: 0.871040723982
Gini: 0.742081447964
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.205076283533: 
Maximum Metrics: Maximum metrics at their respective thresholds

Gains/Lift Table: Avg response rate: 39.53 %


Scoring History: 
Variable Importances: 


## Pritning model objects details as text to try the same process again

In [17]:
print("Model summary  >>> model_object.summary()")
prostate_gbm.summary()

Model summary  >>> model_object.summary()
Model Summary: 


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,8.0,8.0,3087.0,7.0,13.0,9.375,23.0,27.0,25.625


In [18]:
sys.stdout = old_target

In [19]:
!cat /Users/avkashchauhan/Downloads/model_output.txt

Model summary>>> model_object.show()
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  prostate_gbm


ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.036289343297
RMSE: 0.190497620187
LogLoss: 0.170007804527
Mean Per-Class Error: 0.0160045361428
AUC: 0.998865964296
Gini: 0.997731928592
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.487417363665: 
Maximum Metrics: Maximum metrics at their respective thresholds

Gains/Lift Table: Avg response rate: 40.36 %



ModelMetricsBinomial: gbm
** Reported on validation data. **

MSE: 0.161786079676
RMSE: 0.402226403505
LogLoss: 0.483923658542
Mean Per-Class Error: 0.174208144796
AUC: 0.871040723982
Gini: 0.742081447964
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.205076283533: 
Maximum Metrics: Maximum metrics at their respective thresholds

Gains/Lift Table: Avg response rate: 39.53 %


Scoring History: 
Variable Importances: 
